In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,22402
2,Huelva,Confirmados PDIA 14 días,5260
3,Huelva,Tasa PDIA 14 días,"1025,001461503985"
4,Huelva,Confirmados PDIA 7 días,2088
5,Huelva,Total Confirmados,22630
6,Huelva,Curados,11853
7,Huelva,Fallecidos,189


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  22402.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5947.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 301 personas en los últimos 7 días 

Un positivo PCR cada 114 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,22402.0,2088.0,5260.0,513170.0,406.882709,1025.001462,407.0
Huelva-Costa,13581.0,1203.0,3215.0,289548.0,415.475154,1110.351306,242.0
Condado-Campiña,6190.0,665.0,1542.0,156231.0,425.651759,987.000019,133.0
Huelva (capital),5947.0,478.0,1262.0,143837.0,332.320613,877.382037,99.0
Lepe,1803.0,177.0,432.0,27880.0,634.863702,1549.497848,41.0
Isla Cristina,1838.0,136.0,433.0,21393.0,635.721965,2024.026551,29.0
Ayamonte,1059.0,131.0,367.0,21104.0,620.735406,1739.006823,27.0
Sierra de Huelva-Andévalo Central,2257.0,142.0,379.0,67391.0,210.710629,562.389637,27.0
Palma del Condado (La),928.0,80.0,190.0,10801.0,740.672160,1759.096380,17.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,422.0,31.0,109.0,3366.0,920.974450,3238.265003,6.0
Berrocal,16.0,2.0,9.0,303.0,660.066007,2970.297030,0.0
Paymogo,67.0,12.0,33.0,1162.0,1032.702238,2839.931153,1.0
San Bartolomé de la Torre,184.0,35.0,84.0,3761.0,930.603563,2233.448551,5.0
Isla Cristina,1838.0,136.0,433.0,21393.0,635.721965,2024.026551,29.0
Cabezas Rubias,27.0,7.0,14.0,706.0,991.501416,1983.002833,1.0
Palma del Condado (La),928.0,80.0,190.0,10801.0,740.672160,1759.096380,17.0
Ayamonte,1059.0,131.0,367.0,21104.0,620.735406,1739.006823,27.0
Villablanca,128.0,28.0,49.0,2885.0,970.537262,1698.440208,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,198.0,1.0,30.0,4602.0,21.729683,651.890482,0.0,0.033333
Encinasola,116.0,1.0,11.0,1310.0,76.335878,839.694656,0.0,0.090909
Higuera de la Sierra,30.0,2.0,9.0,1291.0,154.918668,697.134005,1.0,0.222222
Berrocal,16.0,2.0,9.0,303.0,660.066007,2970.297030,0.0,0.222222
Aracena,443.0,14.0,57.0,8255.0,169.594185,690.490612,-1.0,0.245614
Nerva,145.0,10.0,38.0,5169.0,193.461018,735.151867,3.0,0.263158
Punta Umbría,588.0,59.0,213.0,15355.0,384.239661,1387.170303,8.0,0.276995
Villalba del Alcor,422.0,31.0,109.0,3366.0,920.974450,3238.265003,6.0,0.284404
Moguer,725.0,50.0,175.0,21867.0,228.655051,800.292678,13.0,0.285714
